In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from lxml import etree
import time
import random
from fake_useragent import UserAgent
import threading
import os
from selenium.common.exceptions import NoSuchWindowException

In [3]:
os.makedirs('Comments')

In [2]:
class getTop250urls():
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
    urls = []
    title = []
    movieurls = dict()
    
    def __init__(self) -> None:
        self.__genUrls()
        self.__getinfo()
        
    def __genUrls(self):
        nums = list(range(0, 10, 1))
        self.urls = [f'https://movie.douban.com/top250?start={str(number*25)}&filter=' for number in nums]
        print('urls生成完成', self.urls)
        
    def __getinfo(self):
        count = 0
        for url in self.urls:
            headers={'User-Agent':UserAgent().random}
            res = requests.get(url=url, headers=headers)
            time.sleep(1)
            res = etree.HTML(res.text)
            p = res.xpath('//div[@class="item"]')
            while len(p)==0:
                res = requests.get(url=url, headers=self.headers)
                res = etree.HTML(res.text)
                p = res.xpath('//div[@class="item"]')
                time.sleep(1)
            for ele in p:
                title = str(ele.xpath('./div[2]/div[1]/a/span[1]/text()')[0])
                movieurl = str(ele.xpath("./div[1]/a/@href")[0])
                count += 1
                self.movieurls[movieurl] = title
                print(count, title, movieurl, sep=' ')
        

getTop250urls()
urls:list = list(getTop250urls.movieurls.keys())
moviedict = getTop250urls.movieurls.copy()
print(len(urls))
print(len(moviedict))

urls生成完成 ['https://movie.douban.com/top250?start=0&filter=', 'https://movie.douban.com/top250?start=25&filter=', 'https://movie.douban.com/top250?start=50&filter=', 'https://movie.douban.com/top250?start=75&filter=', 'https://movie.douban.com/top250?start=100&filter=', 'https://movie.douban.com/top250?start=125&filter=', 'https://movie.douban.com/top250?start=150&filter=', 'https://movie.douban.com/top250?start=175&filter=', 'https://movie.douban.com/top250?start=200&filter=', 'https://movie.douban.com/top250?start=225&filter=']
1 肖申克的救赎 https://movie.douban.com/subject/1292052/
2 霸王别姬 https://movie.douban.com/subject/1291546/
3 阿甘正传 https://movie.douban.com/subject/1292720/
4 泰坦尼克号 https://movie.douban.com/subject/1292722/
5 千与千寻 https://movie.douban.com/subject/1291561/
6 这个杀手不太冷 https://movie.douban.com/subject/1295644/
7 美丽人生 https://movie.douban.com/subject/1292063/
8 星际穿越 https://movie.douban.com/subject/1889243/
9 盗梦空间 https://movie.douban.com/subject/3541415/
10 楚门的世界 https://m

In [3]:
class getRomments():
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
    finishurls = []
    targeturls = []
    def __init__(self) -> None:
        global urls
        cnt = 1
        recommendurls = []
        for ele in urls:
            print(cnt, end=' ')
            recommendurls.extend(self.__getUrlsfromUrl(ele))
            time.sleep(0.2)
            cnt += 1
        urls.extend(recommendurls)
        urls = list(set(urls))
        print('**********************************推荐全部爬取完成**********************************')
        print(len(urls))
        print(len(moviedict))
        

    def __getUrlsfromUrl(self, urlmain:str):
        global moviedict
        resurls = []
        headers={'User-Agent':UserAgent().random}
        res = requests.get(url=urlmain, headers=headers)
        res = etree.HTML(res.text)
        recommend = res.xpath('.//dl')
        for ele in recommend:
            p = str(ele.xpath('./dt/a/@href')[0])
            index = p.index('?')
            p = p[:index]
            resurls.append(p)
            title = str(ele.xpath('./dd/a/text()')[0])
            moviedict[p] = title
        print(getTop250urls.movieurls[urlmain], "推荐已爬取")
        return resurls
# getRomments()

In [4]:
def getrByMovieurl(movieurl:str):
    numbers = list(range(0, 18, 1))
    urls = [f'{movieurl}reviews?start={str(number*20)}' for number in numbers]
    driver = webdriver.Edge()
    moviename = moviedict[movieurl]
    print(f'*******************{moviename}开始爬取评论*******************')
    cnt = 0
    for url in urls:
        try:
            driver.get(url)
            time.sleep(3)
            current_url = driver.current_url
            errorcnt = 0
            while current_url != url and errorcnt<=10:
                time.sleep(5)
                errorcnt += 1
                driver.get(url)
                time.sleep(3)
                current_url = driver.current_url
            if errorcnt >10:
                break

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'review-list')))
            expand_buttons = driver.find_elements(By.XPATH, "//a[@class='unfold']")
            for i in range(0, len(expand_buttons), 1):
                driver.execute_script("arguments[0].click();", expand_buttons[i])
                
            time.sleep(3) #需要等待js执行完  
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="review-content clearfix"]')))
            reviews = driver.find_elements(By.XPATH, '//div[@class="review-content clearfix"]')
            with open(f"Comments/{moviename}.txt", 'a', encoding='utf-8') as f:
                for review in reviews:
                    f.write(review.text)
            f.close()
            cnt += len(reviews)
            indexof = url.index('start=')
            print(f'名字:{moviename}, url页数{url[indexof:]}, 已爬取评论数{cnt}')
        except Exception:
            break
    time.sleep(1)
    driver.close()

In [5]:
def jobsattr(begin, end):
    thisurls = urls[begin:end+1]
    for url in thisurls:
        try:
            getrByMovieurl(url)
        except NoSuchWindowException:
            print('!!!!!!!!!!!')
            return
        except TimeoutError:
            continue

In [6]:
def create_threads(num_threads):
    threads = []
    for i in range(num_threads):
        thread = threading.Thread(target=jobsattr, args=(i*25, i*25+25))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    print("thread over")

create_threads(10)

*******************末代皇帝开始爬取评论*******************
*******************死亡诗社开始爬取评论*******************
*******************肖申克的救赎开始爬取评论*******************
*******************小鞋子开始爬取评论*******************
*******************模仿游戏开始爬取评论*******************
*******************爱在午夜降临前开始爬取评论*******************
*******************哈利·波特与火焰杯开始爬取评论*******************
*******************7号房的礼物开始爬取评论*******************
*******************东邪西毒开始爬取评论*******************
*******************千钧一发开始爬取评论*******************
名字:末代皇帝, url页数start=0, 已爬取评论数19
名字:肖申克的救赎, url页数start=0, 已爬取评论数17
名字:7号房的礼物, url页数start=0, 已爬取评论数19
名字:哈利·波特与火焰杯, url页数start=0, 已爬取评论数20
名字:东邪西毒, url页数start=0, 已爬取评论数19
名字:千钧一发, url页数start=0, 已爬取评论数20
名字:末代皇帝, url页数start=20, 已爬取评论数38
名字:小鞋子, url页数start=0, 已爬取评论数19
名字:爱在午夜降临前, url页数start=0, 已爬取评论数20
名字:模仿游戏, url页数start=0, 已爬取评论数19
名字:死亡诗社, url页数start=0, 已爬取评论数17
名字:肖申克的救赎, url页数start=20, 已爬取评论数37
名字:爱在午夜降临前, url页数start=20, 已爬取评论数40
名字:7号房的礼物, url页数start=20, 已爬取评论数39
名字:哈利·波特与火焰杯, url页数start=20, 

#### 爬取内容
- 每个影片爬取18页list(range(0, 18, 1))，每页大概有15个左右长评论
- 执行耗时:120min
- 爬取有效评论总大小:200MB
- 250个影片左右
#### 计算实际抓取下来的评论数量
- `output.txt`中是上一个代码单元格的输出信息
- 根据打印的消息计算抓取的总长评论数
- 影片数：233；评论数：65256

In [3]:
with open('output.txt', 'r', encoding='utf-8') as f:
    c = (f.readlines())
print(len(c))
c.reverse()
cnt = 0
name = []
for ele in c:
    if not ele.startswith('名字'):
        continue
    p = ele.strip().split(',')
    if p[0] not in name:
        name.append(p[0])
        a = int(p[2][p[2].index('数')+1:])
        cnt += a
        print(p[0], a)
print('评论总数', cnt, '条')
print('影片数:', len(name))

3785
名字:7号房的礼物 352
名字:死亡诗社 346
名字:闻香识女人 346
名字:东邪西毒 75
名字:爱在午夜降临前 139
名字:狩猎 189
名字:贫民窟的百万富翁 18
名字:倩女幽魂 349
名字:英雄本色 352
名字:功夫 234
名字:末代皇帝 352
名字:大话西游之月光宝盒 91
名字:遗愿清单 357
名字:唐伯虎点秋香 77
名字:指环王2：双塔奇兵 350
名字:拯救大兵瑞恩 353
名字:千钧一发 100
名字:绿里奇迹 77
名字:模仿游戏 97
名字:七宗罪 218
名字:阳光灿烂的日子 99
名字:上帝之城 335
名字:玛丽和马克思 339
名字:崖上的波妞 328
名字:完美的世界 322
名字:寻梦环游记 98
名字:大佛普拉斯 120
名字:钢琴家 346
名字:当幸福来敲门 49
名字:加勒比海盗 352
名字:阿凡达 112
名字:背靠背，脸对脸 333
名字:魔女宅急便 191
名字:触不可及 213
名字:哈利·波特与死亡圣器(下) 346
名字:罗生门 58
名字:时空恋旅人 39
名字:新龙门客栈 19
名字:告白 59
名字:教父 321
名字:隐藏人物 179
名字:无耻混蛋 303
名字:少年派的奇幻漂流 350
名字:一一 344
名字:东京教父 353
名字:消失的爱人 197
名字:哪吒闹海 245
名字:西西里的美丽传说 348
名字:未麻的部屋 356
名字:心灵奇旅 346
名字:熔炉 347
名字:鬼子来了 54
名字:朗读者 244
名字:红辣椒 355
名字:九品芝麻官 188
名字:天空之城 157
名字:看不见的客人 358
名字:恋恋笔记本 337
名字:爆裂鼓手 355
名字:谍影重重3 344
名字:神偷奶爸 351
名字:摔跤吧！爸爸 358
名字:大话西游之大圣娶亲 341
名字:借东西的小人阿莉埃蒂 135
名字:哈利·波特与密室 302
名字:弱点 190
名字:疯狂原始人 352
名字:菊次郎的夏天 75
名字:勇敢的心 347
名字:花束般的恋爱 80
名字:釜山行 121
名字:猫鼠游戏 352
名字:虎口脱险 217
名字:音乐之声 353
名字:蜘蛛侠：平行宇宙 59
名字:美国往事 353
名字:小丑 58
名字:萤火之森 119
名字:牯岭街少年

#### 合并为一个文件

In [3]:
files = os.listdir('Comments')
print(files[0], len(files))
def write(filename:str):
    with open(f'Comments/{filename}', 'r', encoding='utf-8') as f:
        content = (f.read())
    f.close()
    with open('评论合并.txt', 'a', encoding='utf-8') as f:
        f.write('\n影片名称:'+filename[:filename.index('.txt')]+'\n')
        f.write(content)
    f.close()

7号房的礼物.txt 233


In [4]:
for file in files:
    print(file)
    write(file)

7号房的礼物.txt
一一.txt
一个叫欧维的男人决定去死.txt
七宗罪.txt
七武士.txt
三傻大闹宝莱坞.txt
三块广告牌.txt
上帝之城.txt
东京教父.txt
东邪西毒.txt
两杆大烟枪.txt
九品芝麻官.txt
乱世佳人.txt
二十二.txt
何以为家.txt
你的名字。.txt
你看起来好像很好吃.txt
侧耳倾听.txt
借东西的小人阿莉埃蒂.txt
倩女幽魂.txt
傲慢与偏见.txt
入殓师.txt
冰川时代.txt
初恋这件小事.txt
剪刀手爱德华.txt
功夫.txt
加勒比海盗.txt
勇敢的心.txt
十二怒汉.txt
千与千寻.txt
千年女优.txt
千钧一发.txt
可可西里.txt
告白.txt
哈利·波特与凤凰社.txt
哈利·波特与密室.txt
哈利·波特与死亡圣器(上).txt
哈利·波特与死亡圣器(下).txt
哈利·波特与火焰杯.txt
哈利·波特与阿兹卡班的囚徒.txt
哈利·波特与魔法石.txt
哈尔的移动城堡.txt
哪吒闹海.txt
唐伯虎点秋香.txt
喜剧之王.txt
城市之光.txt
大佛普拉斯.txt
大红灯笼高高挂.txt
大话西游之大圣娶亲.txt
大话西游之月光宝盒.txt
大闹天宫.txt
大鱼.txt
天使爱美丽.txt
天堂电影院.txt
天空之城.txt
头号玩家.txt
头脑特工队.txt
奇迹男孩.txt
完美的世界.txt
寄生虫.txt
寻梦环游记.txt
射雕英雄传之东成西就.txt
小丑.txt
小偷家族.txt
小森林 冬春篇.txt
小森林 夏秋篇.txt
小鞋子.txt
少年派的奇幻漂流.txt
岁月神偷.txt
崖上的波妞.txt
布达佩斯大饭店.txt
幸福终点站.txt
幽灵公主.txt
弱点.txt
当幸福来敲门.txt
彗星来的那一夜.txt
心灵奇旅.txt
心灵捕手.txt
心迷宫.txt
忠犬八公物语.txt
忠犬八公的故事.txt
怦然心动.txt
怪兽电力公司.txt
恋恋笔记本.txt
恐怖直播.txt
情书.txt
我不是药神.txt
我是山姆.txt
战争之王.txt
房间.txt
拯救大兵瑞恩.txt
指环王1：护戒使者.txt
指环王2：双塔奇兵.txt
指环王3：王者无敌.txt
搏击俱乐部.txt
摔跤吧！爸爸.txt
